In [1]:
import requests
import pandas as pd
import json
from datetime import datetime, timezone
import time

from typing import Dict, List, Optional, Union

from osrs_filter import OSRSItemFilter

from llm.client import ModelConfig, get_chat_response, init_client, remove_thinking_tags
from utils.config import settings
# Configuration
BASE_URL = "https://prices.runescape.wiki/api/v1/osrs"

# IMPORTANT: Set a descriptive User-Agent as required by the API
# Replace with your own descriptive user agent
USER_AGENT = "learning pandas with osrs - @sf1tzp"

# Default headers with proper User-Agent
HEADERS = {
    'User-Agent': USER_AGENT,
    'Accept': 'application/json'
}

print(f"Using User-Agent: {USER_AGENT}")
print(f"Base URL: {BASE_URL}")

Using User-Agent: learning pandas with osrs - @sf1tzp
Base URL: https://prices.runescape.wiki/api/v1/osrs


In [2]:
# Reinitialize with forced reload to clear cached timezone-naive data
print("Reinitializing OSRSItemFilter with fresh data...")
item_filter = OSRSItemFilter(user_agent=USER_AGENT)
item_filter.load_data(force_reload=True)
item_filter.apply_filter(
    margin_min=3000,
    margin_pct_min=3,
    max_hours_since_update=0.25,
)
print(f"Requsting volume data at: {datetime.now()}")
item_filter.load_volume_data(max_items=10000)
print(f"Volume data fetching complete at: {datetime.now()}")

# Example of saving filtered data to files in different formats
print("Saving filtered data to files...")

# Save to Pickle (best for preserving all data types)
item_filter.save("filtered_items.pkl", format="pickle")

Reinitializing OSRSItemFilter with fresh data...
Loading data for filtering...
✅ Computed derived columns
✅ Loaded 4182 items with price data
Starting with 4180 items with price data
After margin >= 3,000 GP: 1083 items
After margin >= 3%: 845 items
After max 0.25h since update: 307 items
Sorted by last_bought_time (descending)
Requsting volume data at: 2025-08-05 17:48:47.726486
Fetching volume data for 307 items...
  Processed 10/307 items...
  Processed 20/307 items...
  Processed 30/307 items...
  Processed 40/307 items...
  Processed 50/307 items...
  Processed 60/307 items...
  Processed 70/307 items...
  Processed 80/307 items...
  Processed 90/307 items...
  Processed 100/307 items...
  Processed 110/307 items...
  Processed 120/307 items...
  Processed 130/307 items...
  Processed 140/307 items...
  Processed 150/307 items...
  Processed 160/307 items...
  Processed 170/307 items...
  Processed 180/307 items...
  Processed 190/307 items...
  Processed 200/307 items...
  Proces

True

In [3]:
# item_filter.save("filtered_items.pkl", format="pickle")

In [7]:
# Initialize a new filter instance
loaded_filter = OSRSItemFilter(user_agent=USER_AGENT)

# Load from the saved pickle file (preserves all data types)
loaded_filter.load_from_file("filtered_items.pkl", format="pickle")

# Current approach: volume_24h + flip_efficiency
loaded_filter = OSRSItemFilter(user_agent=USER_AGENT)
loaded_filter.load_from_file("filtered_items.pkl", format="pickle")
loaded_filter.apply_filter(
    volume_24h_min=150,
    sold_price_max=12000000,
    sort_by=("flip_efficiency", "desc"),
    limit=10
)

with open("over-here!.txt", "w") as file:
    file.write(str(loaded_filter))

# Save with improved formatting for LLM
loaded_filter.save("loaded_filter.json", format="json")


loaded_filter

Loaded 307 items from filtered_items.pkl
✅ Computed derived columns
Loaded 307 items from filtered_items.pkl
✅ Computed derived columns
Starting with 307 items with price data
After sell price <= 12,000,000: 300 items
After 24h volume >= 150: 119 items
Sorted by flip_efficiency (descending)
Limited to top 10 items (from 119)
Data saved successfully to loaded_filter.json



OSRS Price Semantics (counterintuitive):
   • sold_price = what you can BUY at (instant sell order fill price)
   • bought_price = what you can SELL at (instant buy order fill price)
   • margin_gp = bought_price - sold_price = potential profit per item

Trend Values: increasing, decreasing, flat

OSRSItemFilter Results (10 items)
                 name margin_gp margin_pct flip_efficiency  buy_limit sold_price bought_price sold_volume_20m bought_volume_20m avg_sold_price_20m avg_bought_price_20m avg_margin_gp_20m sold_volume_1h bought_volume_1h avg_sold_price_1h avg_bought_price_1h avg_margin_gp_1h sold_price_trend_1h bought_price_trend_1h sold_volume_24h bought_volume_24h avg_sold_price_24h avg_bought_price_24h avg_margin_gp_24h sold_price_trend_24h bought_price_trend_24h sold_price_trend_1w bought_price_trend_1w sold_price_trend_1m bought_price_trend_1m        sold_time      bought_time bought_time_rel sold_time_rel
       Brimstone ring   283,143      5.04%      11,184,148        8

In [8]:
# Prototype LLM interaction using get_generate_response
from llm.client import ModelConfig, get_generate_response
# Configure qwen3:14b with 10k token context
model_config = ModelConfig.create_default("qwen3:14b")
model_config.options.num_ctx = 20000  # 10k token context

# Read the prompt.txt file as system prompt
with open("prompt.txt", "r") as f:
    system_prompt = f.read()



In [9]:

with open("loaded_filter.json", "r") as f:
    content = f.read()

# Use the loaded_filter's string representation as user prompt context
user_prompt = f"{content}"


print("Sending request to LLM...")
print(f"Model: {model_config.name}")
print(f"Context size: {model_config.options.num_ctx}")
print(f"System prompt length: {len(system_prompt)} chars")
print(f"User prompt length: {len(user_prompt)} chars")
print("=" * 50)

# Get response from LLM
response = get_generate_response(
    model_config=model_config,
    system_prompt=system_prompt,
    user_prompt=user_prompt
)

# Clean the response by removing thinking tags
cleaned_response = remove_thinking_tags(response.response)

print("Cleaned response (without thinking tags):")
print("=" * 50)
print(cleaned_response)

# Save the cleaned response to a separate file
with open("llm_trading_analysis_json.md", "w") as f:
    f.write("=== Clean LLM Trading Analysis ===\n")
    f.write(f"Generated at: {datetime.now()}\n")
    f.write(f"Model: {model_config.name}\n")
    f.write("=" * 50 + "\n\n")
    f.write(cleaned_response)

print(f"\nCleaned response saved to 'llm_trading_analysis_clean.md'")
print(f"Cleaned response length: {len(cleaned_response)} characters")

Sending request to LLM...
Model: qwen3:14b
Context size: 20000
System prompt length: 5175 chars
User prompt length: 12307 chars
2025-08-05 17:55:33 [debug    ] retry_backoff                  args=() backoff=1 func=<bound method Client.ps of <ollama._client.Client object at 0x77f2f6562350>> kwargs={} timeout=3600
2025-08-05 17:55:33 [debug    ] connected_to_ollama            url=http://10.0.0.4:8000
2025-08-05 17:55:34 [info     ] sending_generate_request       input_tokens=7749 model=qwen3:14b
2025-08-05 17:55:34 [debug    ] retry_backoff                  args=() backoff=1 func=<bound method Client.generate of <ollama._client.Client object at 0x77f2f6562350>> kwargs={'model': 'qwen3:14b', 'system': 'Let\'s make some short term trades on Old School Runescape! Remember, this is just a game. Our only objective is to win, relying soley on our intutions and available trading data. It\'s fun and it\'s free! This is a great sandbox to experiment with an active dataset.\n\nREMEMBER that in thi

In [10]:

with open("over-here!.txt", "r") as f:
    content = f.read()

# Use the loaded_filter's string representation as user prompt context
user_prompt = f"{content}"


print("Sending request to LLM...")
print(f"Model: {model_config.name}")
print(f"Context size: {model_config.options.num_ctx}")
print(f"System prompt length: {len(system_prompt)} chars")
print(f"User prompt length: {len(user_prompt)} chars")
print("=" * 50)

# Get response from LLM
response = get_generate_response(
    model_config=model_config,
    system_prompt=system_prompt,
    user_prompt=user_prompt
)

# Clean the response by removing thinking tags
cleaned_response = remove_thinking_tags(response.response)

print("Cleaned response (without thinking tags):")
print("=" * 50)
print(cleaned_response)

# Save the cleaned response to a separate file
with open("llm_trading_analysis_txt.md", "w") as f:
    f.write("=== Clean LLM Trading Analysis ===\n")
    f.write(f"Generated at: {datetime.now()}\n")
    f.write(f"Model: {model_config.name}\n")
    f.write("=" * 50 + "\n\n")
    f.write(cleaned_response)

print(f"\nCleaned response saved to 'llm_trading_analysis_txt_clean.md'")
print(f"Cleaned response length: {len(cleaned_response)} characters")

Sending request to LLM...
Model: qwen3:14b
Context size: 20000
System prompt length: 5175 chars
User prompt length: 6931 chars
2025-08-05 18:06:47 [debug    ] retry_backoff                  args=() backoff=1 func=<bound method Client.ps of <ollama._client.Client object at 0x77f2f6562350>> kwargs={} timeout=3600
2025-08-05 18:06:47 [debug    ] connected_to_ollama            url=http://10.0.0.4:8000
2025-08-05 18:06:47 [info     ] sending_generate_request       input_tokens=4318 model=qwen3:14b
2025-08-05 18:06:47 [debug    ] retry_backoff                  args=() backoff=1 func=<bound method Client.generate of <ollama._client.Client object at 0x77f2f6562350>> kwargs={'model': 'qwen3:14b', 'system': 'Let\'s make some short term trades on Old School Runescape! Remember, this is just a game. Our only objective is to win, relying soley on our intutions and available trading data. It\'s fun and it\'s free! This is a great sandbox to experiment with an active dataset.\n\nREMEMBER that in this